# **Task 1: Faster R-CNN**

In [8]:
!pip install deep-sort-realtime

Installing deepsort real time

In [9]:
import torch
import torchvision
import cv2
import numpy as np
from torchvision.transforms import functional as F
from PIL import Image
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Importing required libraries


In [10]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

Using pretrained model resnet and shifting to gpu.

In [11]:
video_path = "/content/football-match.mp4"
output_video_path = "Faster_RCNN_output.mp4"

cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

def preprocess_frame(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_tensor = F.to_tensor(frame_pil).unsqueeze(0).to(device)
    return frame_tensor

The code initializes a video processing pipeline for applying object detection to each frame of a video using Faster R-CNN. It first sets up video capture from an input file and configures an output video writer with the same resolution and frame rate as the input video. Using the preprocess_frame function, each frame is preprocessed by converting it from BGR (used by OpenCV) to RGB format, then into a PIL image, and finally into a PyTorch tensor on the appropriate device (CPU or GPU). This preprocessing ensures compatibility with the model's input requirements. In a loop, each frame is read, preprocessed, and passed through the Faster R-CNN model for object detection. After processing, any detection outputs are drawn onto the frame, which is then saved to the output video file. Finally, the video resources are released to conclude the processing.

In [12]:
last_ball_position = None
frames_since_last_detection = 0

def detect_and_draw_boxes(model, frame):
    global last_ball_position, frames_since_last_detection
    frame_tensor = preprocess_frame(frame)
    with torch.no_grad():
        predictions = model(frame_tensor)
    pred_boxes = predictions[0]['boxes'].cpu().numpy()
    pred_labels = predictions[0]['labels'].cpu().numpy()
    pred_scores = predictions[0]['scores'].cpu().numpy()
    ball_detected = False

    for box, label, score in zip(pred_boxes, pred_labels, pred_scores):
        if score > 0.5 and label in [1]:
            x_min, y_min, x_max, y_max = box
            cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
            text = f"Player {score:.2f}"
            cv2.putText(frame, text, (int(x_min), int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        if score > 0.3 and label == 37:
            x_min, y_min, x_max, y_max = box
            center_x, center_y = (x_min + x_max) / 2, (y_min + y_max) / 2
            last_ball_position = (int(center_x), int(center_y))
            frames_since_last_detection = 0
            ball_detected = True
            cv2.circle(frame, last_ball_position, 5, (0, 0, 255), -1)
            text = f"Ball: {score:.2f}"
            cv2.putText(frame, text, (int(x_min), int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    if not ball_detected and last_ball_position and frames_since_last_detection < 10:
        cv2.circle(frame, last_ball_position, 5, (0, 255, 255), -1)
        cv2.putText(frame, "Predicted Ball", last_ball_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        frames_since_last_detection += 1

    return frame

The detect_and_draw_boxes function integrates object detection into a video processing pipeline, focusing on identifying players and tracking the ball. It accepts a model and frame as inputs, processes the frame using the preprocess_frame function, and passes it through the detection model. For each prediction, the function retrieves bounding box coordinates, labels, and confidence scores. It then differentiates between players and the ball: if the detected object is a player (with a confidence score above 0.5), a green bounding box is drawn, along with the confidence score labeled as "Player." If the detected object is the ball (label 37 with a score above 0.3), a red circle is drawn at the ball's center, with "Ball" and its score displayed above. To handle intermittent detection, the function predicts the ball's position for up to 10 frames following a missed detection by marking its last known position with a yellow circle labeled "Predicted Ball." This setup enables smooth ball tracking and enhances visualization of both players and the ball in sports footage.

In [13]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    result_frame = detect_and_draw_boxes(model, frame)
    out.write(result_frame)
cap.release()
out.release()
print("Processing complete. The output video is saved as:", output_video_path)

Processing complete. The output video is saved as: Faster_RCNN_output.mp4


This segment of code performs the main video processing loop, applying object detection to each frame and saving the output. It continuously reads frames from the video file as long as the capture device is open. For each frame, it applies the detect_and_draw_boxes function to detect and draw bounding boxes around detected objects, such as players and the ball, enhancing the frames with real-time tracking information. The processed frame is then written to an output video file. When all frames are processed, the code releases both the input video capture and output writer resources to free up memory. Finally, it prints a confirmation message indicating that the output video has been saved successfully. This loop is essential for applying model-based object detection frame-by-frame in sports footage.

# **Task 2: Deep SORT**

In [14]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [15]:
tracker = DeepSort(
    max_age=10,
    n_init=3,
    nms_max_overlap=1.0,
    max_cosine_distance=0.3,
    nn_budget=None,
    override_track_class=None,
    embedder="mobilenet",
    half=True,
    bgr=True,
    embedder_gpu=True
)

/usr/local/lib/python3.10/dist-packages/deep_sort_realtime/embedder/embedder_pytorch.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.lo

The tracker initialization sets up an instance of the DeepSort tracker, which uses deep learning-based embeddings for object tracking in video frames. Key parameters configure the tracking behavior:

max_age=10: Specifies that a track will be deleted if it has not been updated for 10 frames, helping manage lost objects.
n_init=3: Requires at least 3 detections to initialize a new track, reducing false positives.\
nms_max_overlap=1.0: Sets the maximum overlap for non-maximum suppression to manage overlapping detections.\
max_cosine_distance=0.3: Controls similarity matching with embeddings; objects with a cosine distance below 0.3 are likely the same.\
nn_budget=None: Sets no limit on the number of features stored for each object, which can help with tracking continuity.\
override_track_class=None: Allows tracking of all classes without restrictions.\
embedder="mobilenet": Uses MobileNet for feature embedding, providing a lightweight model for efficient tracking.\
half=True: Enables half-precision for faster computation.\
bgr=True: Specifies that the input frames are in BGR format, compatible with OpenCV.\
embedder_gpu=True: Enables GPU usage for the embedder, enhancing processing speed.

This tracker is well-suited for multi-object tracking applications in sports analysis, where real-time and continuous tracking of players and other objects is essential.

In [16]:
def process_frame(frame, model):
    frame_tensor = torch.from_numpy(frame/255.0).permute(2,0,1).float().unsqueeze(0).to(device)
    with torch.no_grad():
        predictions = model(frame_tensor)
    detections = []
    for box, score, label in zip(predictions[0]['boxes'], predictions[0]['scores'], predictions[0]['labels']):
        if score > 0.5:
            x1, y1, x2, y2 = box.cpu().numpy()
            if label.item() == 1:
                detections.append(([x1, y1, x2-x1, y2-y1], score.item(), 'player'))
            elif label.item() == 37:
                detections.append(([x1, y1, x2-x1, y2-y1], score.item(), 'ball'))
    return detections

The process_frame function preprocesses a video frame and uses an object detection model to identify and classify objects of interest. It first converts the frame from a NumPy array to a PyTorch tensor, normalizing pixel values between 0 and 1 and rearranging channels to fit the model's input format. The frame tensor is then processed by the model in inference mode (disabling gradient computation for efficiency), and predictions are obtained. For each detected object, the function extracts bounding box coordinates, confidence scores, and labels. If the confidence score exceeds 0.5, the function distinguishes between players and the ball based on label values: 1 for players and 37 for the ball. It then appends each detection as a tuple containing the bounding box, confidence score, and object type (either "player" or "ball") to the detections list. This structure enables efficient object identification and tracking in subsequent steps, making it suitable for applications requiring precise detection of specific objects in sports footage.

In [17]:
cap = cv2.VideoCapture("/content/football-match.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter('Deep_SORT_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    detections = process_frame(frame, model)
    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        if track.get_det_class() == 'player':
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)
        cv2.rectangle(frame,
                      (int(ltrb[0]), int(ltrb[1])),
                      (int(ltrb[2]), int(ltrb[3])),
                      color, 2)
        cv2.putText(frame,
                    f"{track.get_det_class()} {track_id}",
                    (int(ltrb[0]), int(ltrb[1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, color, 2)
    out.write(frame)
cap.release()
out.release()
print("Processing complete. The output video is saved as:", output_video_path)

Processing complete. The output video is saved as: Faster_RCNN_output.mp4


This code captures video frames from a football match, performs object detection, and applies tracking using the Deep SORT algorithm. It first initializes video capture to read frames and sets up an output writer with the same resolution and frame rate as the input. In the main loop, each frame is read and processed for detections using the process_frame function, which identifies players and the ball. The detected objects are then passed to the tracker.update_tracks method, which assigns each detected object a unique track_id and manages their positions across frames.

For each confirmed track, the code retrieves the bounding box coordinates and assigns a color: green for players and red for the ball. It then draws a labeled bounding box around each tracked object, displaying the class name and track ID. The processed frame is written to the output video. Once all frames are processed, resources for video capture and writing are released, and the final video, now containing tracked players and ball movement, is saved as Deep_SORT_output.mp4. This process allows for continuous and real-time tracking of key objects, making it useful for sports analytics applications.

# **Object Tracking architechture:**

![Architechture diagram assignment 3](...)

# Key Components of the Object Tracking Architecture

This object tracking architecture combines **object detection** with **object tracking** using Faster R-CNN, Deep SORT, the Kalman Filter, and the Hungarian Algorithm. Here’s an overview of each component:

## 1. Object Detection (Faster R-CNN)
Faster R-CNN is used to detect objects in each video frame. The model produces bounding boxes, labels, and confidence scores for objects, identifying key elements such as players and the ball. These detections serve as inputs for the tracking algorithm in subsequent frames.

## 2. Deep SORT Tracker
Deep SORT (Simple Online and Realtime Tracking) extends the SORT (Simple Online Realtime Tracking) algorithm with a deep learning-based feature embedding model (MobileNet in this setup) for object re-identification. This enables the tracker to associate detections over time, even when objects temporarily disappear or overlap.

## 3. Kalman Filter for State Estimation
The Kalman Filter is used to predict and update the object’s state, maintaining the location and velocity of each detected object between frames. It consists of two main steps:

### Predict Step
$\hat{x}_{k|k-1} = F \cdot \hat{x}_{k-1|k-1}$

$P_{k|k-1} = F \cdot P_{k-1|k-1} \cdot F^T + Q$

Where:
- $\hat{x}_{k|k-1}$ is the predicted state (position, velocity) at time $k$
- $F$ is the state transition model
- $P_{k|k-1}$ is the predicted covariance
- $Q$ is the process noise

### Update Step
$K_k = P_{k|k-1} \cdot H^T \cdot (H \cdot P_{k|k-1} \cdot H^T + R)^{-1}$

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_k \cdot (z_k - H \cdot \hat{x}_{k|k-1})$

$P_{k|k} = (I - K_k \cdot H) \cdot P_{k|k-1}$

Where:
- $K_k$ is the Kalman gain
- $H$ is the measurement model
- $R$ is the measurement noise
- $z_k$ is the measurement (detection) at time $k$



This update corrects the prediction based on the actual observation.

## 4. Hungarian Algorithm for Data Association
The Hungarian Algorithm assigns detections to existing tracked objects. In each frame, a cost matrix is constructed where each element represents the distance (often the cosine distance in feature space) between detected objects and tracked objects. The algorithm minimizes the overall cost, finding the optimal assignment of detections to tracks.

This process helps manage scenarios where objects may occlude or appear in different parts of the frame. If no match is found within a threshold, new tracks are initiated or existing tracks are terminated.
